# Create a clickable map of CDEC stations with the Delta

Display time series for selected station.
Development notebook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import geopandas as gpd


from cdec_maps import cdec

In [ ]:
import hvplot.pandas
import holoviews as hv
from holoviews import opts

In [ ]:
def get_all_stations():
    c = cdec.Reader()
    daily_stations = c.read_daily_stations()
    realtime_stations = c.read_realtime_stations()
    all_stations = daily_stations.merge(realtime_stations, how='outer')
    return all_stations


def convert_to_gpd(stations):
    stations = gpd.GeoDataFrame(
        stations, geometry=gpd.points_from_xy(stations.Longitude, stations.Latitude))
    stations = stations.set_crs(epsg=4326)
    return stations


def station_within_delta(stations):
    delta_boundary = gpd.read_file('./Delta_Simplified.geojson')
    stations_delta = stations[stations.within(delta_boundary.geometry[0])]
    return stations_delta

In [ ]:
# utility function to load all stations meta data into a single table
def station_meta_table(stations):
    reader = cdec.Reader()
    dfmeta = pd.DataFrame(stations.ID).reset_index(drop=True)
    for station in stations:
        sid = station.ID
        dflist = reader.read_station_meta_info(sid)
        dflist[0]


def add_station_meta_info(stations):
    reader = cdec.Reader()
    stations = stations.set_index('ID')
    result = pd.DataFrame()
    for station in stations.iterrows():
        station_id = station[0]
        dflist = reader.read_station_meta_info(station_id)
        result = pd.concat([result, stations.join(
            dflist[1].assign(ID=station_id).set_index('ID'))])
    # don't know why this astype is needed ?!
    return result.dropna().astype(dtype={'Sensor Number': 'int'})

In [ ]:
all_stations=get_all_stations()
all_stations.head(1)

In [ ]:
reader = cdec.Reader()

In [ ]:
meta_info_list=[reader.read_station_meta_info(station_id)[1].assign(ID=station_id).set_index('ID') for station_id in all_stations.ID]

In [ ]:
len(meta_info_list), len(all_stations)

In [ ]:
dfmeta=pd.concat(meta_info_list)

In [ ]:
dfmeta['Sensor Number'].astype(int)

In [ ]:
dfmeta['Sensor Number'].unique()

In [ ]:
delta_stations = station_within_delta(convert_to_gpd(get_all_stations()))
delta_stations

In [ ]:
dfmeta = add_station_meta_info(delta_stations)
dfmeta.head(1)

In [ ]:
sensor_descriptions=dfmeta['Sensor Description'].sort_values().unique()

In [ ]:
dfmeta[['Sensor Description','Sensor Number']]

In [ ]:
map = delta_stations.hvplot.points(
    geo=True, tiles='OSM', frame_width=400, hover_cols='all')

In [ ]:
stn_id = 'BAC'
meta_row = dfmeta.loc[stn_id].iloc[0, :]
meta_row

In [ ]:
reader = cdec.Reader()
data = reader.read_station_data(stn_id, meta_row['Sensor Number'], cdec.get_duration_code(
    meta_row['Duration']), start='2021-10-01', end='')
data.head(1)

In [ ]:
crv1 = hv.Curve(data.loc[:, 'VALUE']).opts(ylabel=meta_row['Plot'])
crv1

In [ ]:
stn_id = 'BAC'


def show_plot(stn_id):
    crv_list = []
    for id, meta_row in dfmeta.loc[stn_id].iterrows():
        data = reader.read_station_data(stn_id, meta_row['Sensor Number'], cdec.get_duration_code(
            meta_row['Duration']), start='2021-10-01', end='')
        crv_list.append(hv.Curve(data.loc[:, 'VALUE']).redim(
            VALUE=meta_row['Plot']))
    return hv.Layout(crv_list).cols(1).opts(opts.Curve(width=700))

In [ ]:
import param
import panel as pn
pn.extension()

In [ ]:
import dask

In [ ]:
pd.Timestamp.now().floor('90D')

In [ ]:
dr=param.CalendarDateRange(default=(pd.Timestamp.now().floor('90D'),pd.Timestamp.now().ceil('1D')))

In [ ]:
class CDECPlotter(param.Parameterized):
    selected = param.List(
        default=[0], doc='Selected node indices to display in plot')
    date_range = param.DateRange(default=(pd.Timestamp.now().floor('90D'),pd.Timestamp.now().ceil('1D')))
    def __init__(self, stations, station_meta_info, **kwargs):
        super().__init__(**kwargs)
        self.stations = stations
        self.stations_meta = station_meta_info
        self.points_map = self.stations.hvplot.points(geo=True, tiles='CartoLight',  # c='WELL_TYPE',
                                                      frame_height=400, frame_width=300,
                                                      fill_alpha=0.9, line_alpha=0.4,
                                                      hover_cols=['index', 'ID', 'Station'])
        self.points_map = self.points_map.opts(opts.Points(tools=['tap', 'hover'], size=5,
                                                           nonselection_color='red', nonselection_alpha=0.6,
                                                           active_tools=['wheel_zoom']))
        # create a selection and add it to a dynamic map calling back show_ts
        self.select_stream = hv.streams.Selection1D(
            source=self.points_map, index=[0])
        self.select_stream.add_subscriber(self.set_selected)
        self.reader = cdec.Reader()

    def set_selected(self, index):
        if index is None or len(index) == 0:
            pass  # keep the previous selections
        else:
            self.selected = index

    @param.depends('selected')
    def show_meta(self):
        index = self.selected
        if index is None or len(index) == 0:
            index = self.selected
        # Use only the first index in the array
        first_index = index[0]
        dfselected = self.stations.iloc[first_index, :]
        stn_id = dfselected['ID']
        return pn.widgets.DataFrame(self.stations_meta.loc[stn_id, :].drop(columns='geometry'))

    @dask.delayed
    def get_sensor_data(self, stn_id, sensor_number, duration_code, start, end):
        return self.reader.read_station_data(stn_id, sensor_number, duration_code, start, end)

    def get_all_sensor_data(self, stn_id):
        return [self.get_sensor_data(stn_id,
                                meta_row['Sensor Number'],
                                cdec.get_duration_code(meta_row['Duration']),
                                start=self.date_range[0].strftime('%Y-%m-%d'),
                                end=self.date_range[1].strftime('%Y-%m-%d'))
                for id, meta_row in self.stations_meta.loc[stn_id].iterrows()]

    @param.depends('selected')
    def show_ts(self):
        index = self.selected
        if index is None or len(index) == 0:
            index = self.selected
        # Use only the first index in the array
        first_index = index[0]
        dfselected = self.stations.iloc[first_index, :]
        stn_id = dfselected['ID']
        stn_name = dfselected['Station']
        # get data for each row and make a curve
        crv_list = []
        data_array = dask.compute(*self.get_all_sensor_data(stn_id))
        for index, (id, meta_row) in enumerate(self.stations_meta.loc[stn_id].iterrows()):
            data = data_array[index]
            crv_list.append(hv.Curve(data.loc[:, 'VALUE']).redim(VALUE=meta_row['Plot'])
                            .opts(title=f'Sensor: Description: {meta_row["Sensor Description"]} {meta_row["Duration"]}'))
        layout = hv.Layout(crv_list).cols(1).opts(opts.Curve(width=900))
        return layout.opts(title=f'{stn_id}: {stn_name}')

In [ ]:
plotter = CDECPlotter(delta_stations, dfmeta)

In [ ]:
pn.Column(pn.Row(pn.widgets.DateRangeSlider(plotter.date_range)), pn.Row(plotter.points_map, plotter.show_meta), pn.Row(plotter.show_ts))